In [1]:
# Import necessary libraries
import os
import pandas as pd

# 1. Accuracy Results

In [2]:
# Function for extracting accuracy results

def accuracy_results():
    # Define the base directory path where the Excel files are located
    directory_path = '../../data/accuracy_results/'

    # Define the subdirectories for different models
    models = ['BASELINE', 'GRU', 'LSTM', 'SARIMA', 'TBATS']

    # Define the data locations for NRMSE and NMAE in each forecast category
    data_locations = {
        '14 days':
            {'NRMSE': {'14 Days Training': (3, 'B:K'), 'Training:Testing = 75:25': (3, 'R:AA'), 'Training:Testing = 50:50': (3, 'AH:AQ')},
            'NMAE': {'14 Days Training': (4, 'B:K'), 'Training:Testing = 75:25': (4, 'R:AA'), 'Training:Testing = 50:50': (4, 'AH:AQ')}},
        '7 days':
            {'NRMSE': {'14 Days Training': (8, 'B:K'), 'Training:Testing = 75:25': (8, 'R:AA'), 'Training:Testing = 50:50': (8, 'AH:AQ')},
            'NMAE': {'14 Days Training': (9, 'B:K'), 'Training:Testing = 75:25': (9, 'R:AA'), 'Training:Testing = 50:50': (9, 'AH:AQ')}},
        '3 days':
            {'NRMSE': {'14 Days Training': (13, 'B:K'), 'Training:Testing = 75:25': (13, 'R:AA'), 'Training:Testing = 50:50': (13, 'AH:AQ')},
            'NMAE': {'14 Days Training': (14, 'B:K'), 'Training:Testing = 75:25': (14, 'R:AA'), 'Training:Testing = 50:50': (14, 'AH:AQ')}},
        '1 day':
            {'NRMSE': {'14 Days Training': (18, 'B:K'), 'Training:Testing = 75:25': (18, 'R:AA'), 'Training:Testing = 50:50': (18, 'AH:AQ')},
            'NMAE': {'14 Days Training': (19, 'B:K'), 'Training:Testing = 75:25': (19, 'R:AA'), 'Training:Testing = 50:50': (19, 'AH:AQ')}},
    }

    # Initialize a list to hold the combined NRMSE and NMAE data
    combined_data = []

    # Iterate over each training method directory
    for model in models:
        # Construct the full path to the model directory
        model_path = os.path.join(directory_path, model)
        
        # List all Excel files in the directory
        excel_files = [f for f in os.listdir(model_path) if f.endswith('.xlsx')]
        
        # Iterate over each Excel file
        for file in excel_files:
            # Construct the full file path
            file_path = os.path.join(model_path, file)
            
            # Iterate over each forecast category to extract NRMSE and NMAE data
            for forecast_period, metrics in data_locations.items():
                for metric, training_methods in metrics.items():
                    for training_method, (row, columns) in training_methods.items():
                        try:
                            # Load the Excel file data for the specified range
                            df = pd.read_excel(file_path, usecols=columns, skiprows=row, nrows=1, header=None)
                            # Flatten the data and add it to the combined data list
                            values = df.values.flatten().tolist()
                            combined_data.extend([(model, forecast_period, metric, training_method, value) for value in values])
                        except Exception as e:
                            print(f"Error reading {file_path} for forecast period {forecast_period}, metric {metric}: {e}")

    # Convert the combined data list to a DataFrame
    combined_df = pd.DataFrame(combined_data, columns=['Model', 'Forecast Period', 'Metric', 'Training Method', 'Value'])
    return combined_df

In [3]:
# Extract accuracy results
combined_df_accuracy = accuracy_results()


## Filter dataset for forecast period and training method so that the measurements are not impacted by outliers
combined_df_accuracy_filtered = combined_df_accuracy[combined_df_accuracy['Value'] < 5]

In [4]:
# Group by particular category
demand_model_accuracies = combined_df_accuracy.groupby(by=['Metric', 'Model']).mean(numeric_only=True).round(3)
forecast_period_accuracies = combined_df_accuracy_filtered.groupby(by=['Metric', 'Forecast Period']).mean(numeric_only=True).round(3)
training_method_accuracies = combined_df_accuracy_filtered.groupby(by=['Metric', 'Training Method']).mean(numeric_only=True).round(3)

In [5]:
# Sort by value
demand_model_accuracies.sort_values(by=['Metric', 'Value'], inplace=True)
forecast_period_accuracies.sort_values(by=['Metric', 'Value'], inplace=True)
training_method_accuracies.sort_values(by=['Metric', 'Value'], inplace=True)

In [27]:
# Display results
print(demand_model_accuracies)
print(forecast_period_accuracies)
print(training_method_accuracies)

                      Value
Metric Model               
NMAE   SARIMA         0.788
       BASELINE       1.161
       GRU            1.827
       LSTM           1.983
       TBATS      94717.229
NRMSE  SARIMA         1.105
       BASELINE       1.468
       GRU            2.186
       LSTM           2.302
       TBATS     139330.911
                        Value
Metric Forecast Period       
NMAE   7 days           1.107
       14 days          1.156
       3 days           1.160
       1 day            1.234
NRMSE  7 days           1.392
       14 days          1.404
       3 days           1.418
       1 day            1.512
                                 Value
Metric Training Method                
NMAE   14 Days Training          1.106
       Training:Testing = 50:50  1.191
       Training:Testing = 75:25  1.195
NRMSE  14 Days Training          1.369
       Training:Testing = 75:25  1.435
       Training:Testing = 50:50  1.489


In [7]:
# Record accuracy results
def main():
    # Make the directory for the results
    path = '../../data/results_summary'
    if os.path.isdir(path):
        pass
    else:
        os.makedirs(path)
    file_name = path + '/Accuracy_Results_Summary.xlsx'
    with pd.ExcelWriter(file_name) as writer:
        demand_model_accuracies.to_excel(writer, sheet_name='demand_model_accuracy')
        forecast_period_accuracies.to_excel(writer, sheet_name='forecast_period_accuracy')
        training_method_accuracies.to_excel(writer, sheet_name='training_method_accuracy')
        
        worksheet1 = writer.sheets['demand_model_accuracy']
        worksheet2 = writer.sheets['forecast_period_accuracy']
        worksheet3 = writer.sheets['training_method_accuracy']
        
        # Adjust the column width
        worksheet1.set_column(1, 2, 10)
        worksheet2.set_column(1, 1, 15)
        worksheet2.set_column(2, 2, 8)
        worksheet3.set_column(1, 1, 23)
        worksheet3.set_column(2, 2, 8)
        
        # Adjust cell formating
        format1 = writer.book.add_format({'border': 1})
        worksheet1.conditional_format(0, 0, 10, 2, {'type': 'no_blanks', 'format': format1})
        worksheet2.conditional_format(0, 0, 10, 2, {'type': 'no_blanks', 'format': format1})
        worksheet3.conditional_format(0, 0, 10, 2, {'type': 'no_blanks', 'format': format1})

main()

# 2. Execution Time Results

In [8]:
# Function for extracting execution time results

def execution_time_results():
    # Define the base directory path where the Excel files are located
    directory_path = '../../data/execution_time_results/'

    # Define the subdirectories for different models
    models = ['BASELINE', 'GRU', 'LSTM', 'SARIMA', 'TBATS']

    # Define the data locations for execution time values in each forecast category
    data_locations = {
        '14 days':
            {'Time (s)': {'14 Days Training': (3, 'B:K'), 'Training:Testing = 75:25': (3, 'R:AA'), 'Training:Testing = 50:50': (3, 'AH:AQ')}},
        '7 days':
            {'Time (s)': {'14 Days Training': (7, 'B:K'), 'Training:Testing = 75:25': (7, 'R:AA'), 'Training:Testing = 50:50': (7, 'AH:AQ')}},
        '3 days':
            {'Time (s)': {'14 Days Training': (11, 'B:K'), 'Training:Testing = 75:25': (11, 'R:AA'), 'Training:Testing = 50:50': (11, 'AH:AQ')}},
        '1 day':
            {'Time (s)': {'14 Days Training': (15, 'B:K'), 'Training:Testing = 75:25': (15, 'R:AA'), 'Training:Testing = 50:50': (15, 'AH:AQ')}},
    }

    # Initialize a list to hold the combined execution time data
    combined_data = []

    # Iterate over each training method directory
    for model in models:
        # Construct the full path to the model directory
        model_path = os.path.join(directory_path, model)
        
        # List all Excel files in the directory
        excel_files = [f for f in os.listdir(model_path) if f.endswith('.xlsx')]
        
        # Iterate over each Excel file
        for file in excel_files:
            # Construct the full file path
            file_path = os.path.join(model_path, file)
            
            # Iterate over each forecast category to extract execution time data
            for forecast_period, metrics in data_locations.items():
                for metric, training_methods in metrics.items():
                    for training_method, (row, columns) in training_methods.items():
                        try:
                            # Load the Excel file data for the specified range
                            df = pd.read_excel(file_path, usecols=columns, skiprows=row, nrows=1, header=None)
                            # Flatten the data and add it to the combined data list
                            values = df.values.flatten().tolist()
                            combined_data.extend([(model, forecast_period, metric, training_method, value) for value in values])
                        except Exception as e:
                            print(f"Error reading {file_path} for forecast period {forecast_period}, metric {metric}: {e}")

    # Convert the combined data list to a DataFrame
    combined_df = pd.DataFrame(combined_data, columns=['Model', 'Forecast Period', 'Metric', 'Training Method', 'Value'])
    return combined_df

In [9]:
# Extract execution time results
combined_df_execution_time = execution_time_results()

In [10]:
# Group by particular category
demand_model_execution_times = combined_df_execution_time.groupby(by=['Metric', 'Model']).mean(numeric_only=True).round(3)
forecast_period_execution_times = combined_df_execution_time.groupby(by=['Metric', 'Forecast Period']).mean(numeric_only=True).round(3)
training_method_execution_times = combined_df_execution_time.groupby(by=['Metric', 'Training Method']).mean(numeric_only=True).round(3)

In [11]:
# Sort by value
demand_model_execution_times.sort_values(by='Value', inplace=True)
forecast_period_execution_times.sort_values(by='Value', inplace=True)
training_method_execution_times.sort_values(by='Value', inplace=True)

In [28]:
# Display results
print(demand_model_execution_times)
print(forecast_period_execution_times)
print(training_method_execution_times)

                    Value
Metric   Model           
Time (s) BASELINE   0.001
         SARIMA     0.926
         LSTM      15.025
         GRU       15.047
         TBATS     21.249
                           Value
Metric   Forecast Period        
Time (s) 3 days            6.859
         1 day             7.734
         7 days           10.466
         14 days          16.739
                                    Value
Metric   Training Method                 
Time (s) Training:Testing = 50:50   8.831
         14 Days Training          10.140
         Training:Testing = 75:25  12.378


In [13]:
# Record execution time results
def main():
    # Make the directory for the results
    path = '../../data/results_summary'
    if os.path.isdir(path):
        pass
    else:
        os.makedirs(path)
    file_name = path + '/Execution_Time_Results_Summary.xlsx'
    with pd.ExcelWriter(file_name) as writer:
        demand_model_execution_times.to_excel(writer, sheet_name='demand_model_execution_time')
        forecast_period_execution_times.to_excel(writer, sheet_name='forecast_period_execution_time')
        training_method_execution_times.to_excel(writer, sheet_name='training_method_execution_time')
        
        worksheet1 = writer.sheets['demand_model_execution_time']
        worksheet2 = writer.sheets['forecast_period_execution_time']
        worksheet3 = writer.sheets['training_method_execution_time']
        
        # Adjust the column width
        worksheet1.set_column(1, 2, 10)
        worksheet2.set_column(1, 1, 15)
        worksheet2.set_column(2, 2, 10)
        worksheet3.set_column(1, 1, 23)
        worksheet3.set_column(2, 2, 10)
        
        # Adjust cell formating
        format1 = writer.book.add_format({'border': 1})
        worksheet1.conditional_format(0, 0, 10, 2, {'type': 'no_blanks', 'format': format1})
        worksheet2.conditional_format(0, 0, 10, 2, {'type': 'no_blanks', 'format': format1})
        worksheet3.conditional_format(0, 0, 10, 2, {'type': 'no_blanks', 'format': format1})

main()

# 3. Demand-Proportionality Results

In [14]:
# Function for extracting demand-proportionality results

def demand_proportionality_results():
    # Construct the full file path
    directory_path = '../../data/demand_proportionality_results/'

    # Define the data locations for demand-proportionality in each forecast category
    data_locations = {
        '28 day':
            {'Pearson Correlation Coefficient': {'Flat Energy': (2, 'B:K'), 'Time of Use': (3, 'B:K'), 'Range': (4, 'B:K'), 'Percentile': (5, 'B:K')}},
        '14 days':
            {'Pearson Correlation Coefficient': {'Flat Energy': (9, 'B:K'), 'Time of Use': (10, 'B:K'), 'Range': (11, 'B:K'), 'Percentile': (12, 'B:K')}},
        '7 days':
            {'Pearson Correlation Coefficient': {'Flat Energy': (16, 'B:K'), 'Time of Use': (17, 'B:K'), 'Range': (18, 'B:K'), 'Percentile': (19, 'B:K')}},
        '3 days':
            {'Pearson Correlation Coefficient': {'Flat Energy': (23, 'B:K'), 'Time of Use': (24, 'B:K'), 'Range': (25, 'B:K'), 'Percentile': (26, 'B:K')}},
        '2 day':
            {'Pearson Correlation Coefficient': {'Flat Energy': (30, 'B:K'), 'Time of Use': (31, 'B:K'), 'Range': (32, 'B:K'), 'Percentile': (33, 'B:K')}},
        '1 day':
            {'Pearson Correlation Coefficient': {'Flat Energy': (37, 'B:K'), 'Time of Use': (38, 'B:K'), 'Range': (39, 'B:K'), 'Percentile': (40, 'B:K')}},
    }

    # Initialize a list to hold the combined demand-proportionality data
    combined_data = []
        
    # List all Excel files in the directory
    excel_files = [f for f in os.listdir(directory_path) if f.endswith('.xlsx')]
    
    # Iterate over each Excel file
    for file in excel_files:
        # Construct the full file path
        file_path = os.path.join(directory_path, file)
        
        # Iterate over each forecast category to extract demand-proportionality data
        for test_period, metrics in data_locations.items():
            for metric, pricing_methods in metrics.items():
                for pricing_method, (row, columns) in pricing_methods.items():
                    try:
                        # Load the Excel file data for the specified range
                        df = pd.read_excel(file_path, usecols=columns, skiprows=row, nrows=1, header=None)
                        # Flatten the data and add it to the combined data list
                        values = df.values.flatten().tolist()
                        combined_data.extend([(test_period, metric, pricing_method, value) for value in values])
                    except Exception as e:
                        print(f"Error reading {file_path} for test period {test_period}, pricing method {pricing_method}: {e}")

    # Convert the combined data list to a DataFrame
    combined_df = pd.DataFrame(combined_data, columns=['Test Period', 'Metric', 'Pricing Method', 'Value'])
    return combined_df

In [15]:
# Extract demand-proportionality results
combined_df_demand_proportionality = demand_proportionality_results()

In [16]:
# Group by particular category
pricing_method_demand_proportionalities = combined_df_demand_proportionality.groupby(by=['Metric', 'Pricing Method']).mean(numeric_only=True).round(3)
test_period_demand_proportionalities = combined_df_demand_proportionality.groupby(by=['Metric', 'Test Period']).mean(numeric_only=True).round(3)

In [17]:
# Sort by value
pricing_method_demand_proportionalities.sort_values(by='Value', inplace=True)
test_period_demand_proportionalities.sort_values(by='Value', inplace=True)

In [29]:
# Display results
print(pricing_method_demand_proportionalities)
print(test_period_demand_proportionalities)

                                                Value
Metric                          Pricing Method       
Pearson Correlation Coefficient Flat Energy     0.000
                                Time of Use     0.555
                                Percentile      0.918
                                Range           0.982
                                             Value
Metric                          Test Period       
Pearson Correlation Coefficient 14 days      0.607
                                28 day       0.607
                                7 days       0.611
                                2 day        0.617
                                3 days       0.619
                                1 day        0.623


In [19]:
# Record demand-proportionality results
def main():
    # Make the directory for the results
    path = '../../data/results_summary'
    if os.path.isdir(path):
        pass
    else:
        os.makedirs(path)
    file_name = path + '/Demand_Proportionality_Results_Summary.xlsx'
    with pd.ExcelWriter(file_name) as writer:
        pricing_method_demand_proportionalities.to_excel(writer, sheet_name='price_demand_proportionality')
        test_period_demand_proportionalities.to_excel(writer, sheet_name='period_demand_proportionality')
        
        worksheet1 = writer.sheets['price_demand_proportionality']
        worksheet2 = writer.sheets['period_demand_proportionality']
        
        # Adjust the column width
        worksheet1.set_column(0, 0, 30)
        worksheet1.set_column(1, 1, 15)
        worksheet1.set_column(2, 2, 8)
        worksheet2.set_column(0, 0, 30)
        worksheet2.set_column(1, 1, 15)
        worksheet2.set_column(2, 2, 8)
        
        # Adjust cell formating
        format1 = writer.book.add_format({'border': 1})
        worksheet1.conditional_format(0, 0, 10, 2, {'type': 'no_blanks', 'format': format1})
        worksheet2.conditional_format(0, 0, 10, 2, {'type': 'no_blanks', 'format': format1})

main()

# 4. Revenue Results

In [20]:
# Function for extracting revenue results

def revenue_results():
    # Construct the full file path
    directory_path = '../../data/revenue_results/'

    # Define the data locations for revenue in each forecast category
    data_locations = {
        '28 day':
            {'Revenue': {'Flat Energy': (2, 'B:K'), 'Time of Use': (3, 'B:K'), 'Range': (4, 'B:K'), 'Percentile': (5, 'B:K')}},
        '14 days':
            {'Revenue': {'Flat Energy': (9, 'B:K'), 'Time of Use': (10, 'B:K'), 'Range': (11, 'B:K'), 'Percentile': (12, 'B:K')}},
        '7 days':
            {'Revenue': {'Flat Energy': (16, 'B:K'), 'Time of Use': (17, 'B:K'), 'Range': (18, 'B:K'), 'Percentile': (19, 'B:K')}},
        '3 days':
            {'Revenue': {'Flat Energy': (23, 'B:K'), 'Time of Use': (24, 'B:K'), 'Range': (25, 'B:K'), 'Percentile': (26, 'B:K')}},
        '2 day':
            {'Revenue': {'Flat Energy': (30, 'B:K'), 'Time of Use': (31, 'B:K'), 'Range': (32, 'B:K'), 'Percentile': (33, 'B:K')}},
        '1 day':
            {'Revenue': {'Flat Energy': (37, 'B:K'), 'Time of Use': (38, 'B:K'), 'Range': (39, 'B:K'), 'Percentile': (40, 'B:K')}},
    }

    # Initialize a list to hold the combined revenue data
    combined_data = []
        
    # List all Excel files in the directory
    excel_files = [f for f in os.listdir(directory_path) if f.endswith('.xlsx')]
    
    # Iterate over each Excel file
    for file in excel_files:
        # Construct the full file path
        file_path = os.path.join(directory_path, file)
        
        # Iterate over each forecast category to extract revenue data
        for test_period, metrics in data_locations.items():
            for metric, pricing_methods in metrics.items():
                for pricing_method, (row, columns) in pricing_methods.items():
                    try:
                        # Load the Excel file data for the specified range
                        df = pd.read_excel(file_path, usecols=columns, skiprows=row, nrows=1, header=None)
                        # Flatten the data and add it to the combined data list
                        values = df.values.flatten().tolist()
                        combined_data.extend([(test_period, metric, pricing_method, value) for value in values])
                    except Exception as e:
                        print(f"Error reading {file_path} for test period {test_period}, pricing method {pricing_method}: {e}")

    # Convert the combined data list to a DataFrame
    combined_df = pd.DataFrame(combined_data, columns=['Test Period', 'Metric', 'Pricing Method', 'Value'])
    return combined_df

In [21]:
# Extract revenue results
combined_df_revenue = revenue_results()

In [22]:
# Group by particular category
pricing_method_revenues = combined_df_revenue.groupby(by=['Metric', 'Pricing Method']).mean(numeric_only=True).round(2)
pricing_method_period_revenues = combined_df_revenue.groupby(by=['Metric', 'Test Period', 'Pricing Method']).mean(numeric_only=True).round(2)

In [23]:
# Sort by value
pricing_method_revenues.sort_values(by='Value', inplace=True, ascending=False)
pricing_method_period_revenues.sort_values(by=['Test Period', 'Value'], inplace=True, ascending=False)

In [30]:
# Display results
print(pricing_method_revenues)
print(pricing_method_period_revenues)

                         Value
Metric  Pricing Method        
Revenue Time of Use     258.36
        Percentile      257.77
        Range           244.33
        Flat Energy     243.66
                                     Value
Metric  Test Period Pricing Method        
Revenue 7 days      Time of Use     194.48
                    Percentile      193.96
                    Range           184.61
                    Flat Energy     183.36
        3 days      Time of Use      83.25
                    Percentile       82.92
                    Range            79.77
                    Flat Energy      78.37
        28 day      Time of Use     799.43
                    Percentile      798.03
                    Flat Energy     754.26
                    Range           753.11
        2 day       Time of Use      53.86
                    Percentile       53.60
                    Range            51.73
                    Flat Energy      50.68
        14 days     Time of Use     393.

In [25]:
# Record revenue results
def main():
    # Make the directory for the results
    path = '../../data/results_summary'
    if os.path.isdir(path):
        pass
    else:
        os.makedirs(path)
    file_name = path + '/Revenue_Results_Summary.xlsx'
    with pd.ExcelWriter(file_name) as writer:
        pricing_method_revenues.to_excel(writer, sheet_name='price_method_revenue')
        pricing_method_period_revenues.to_excel(writer, sheet_name='test_period_revenue')
        
        worksheet1 = writer.sheets['price_method_revenue']
        worksheet2 = writer.sheets['test_period_revenue']
        
        # Adjust the column width
        worksheet1.set_column(1, 1, 15)
        worksheet1.set_column(2, 2, 8)
        worksheet2.set_column(1, 2, 15)
        worksheet2.set_column(3, 3, 8)
        
        # Adjust cell formating
        format1 = writer.book.add_format({'border': 1})
        worksheet1.conditional_format(0, 0, 10, 3, {'type': 'no_blanks', 'format': format1})
        worksheet2.conditional_format(0, 0, 24, 3, {'type': 'no_blanks', 'format': format1})

main()